# Findings and Caveats

In Fall 2018, international students contributed an estimated 199 million to institutions of higher education in Connecticut. This includes money that international students pay out of pocket for their institutions, as well as money that covers their tuition through grants and the like, as is more common amoung doctoral students, i.e. it does not discriminate based on funding source, as is consistent with university budgeting practice. This analysis does not not include housing costs.

Caveat: Four schools that reported international enrollments — totaling 35 students between them in Fall 2018 — but no associated tuition cost was available. The enrollment data does not include students solely enrolled in ESL courses.

The figure was obtained by multiplying the the National Council for Education Statistics' IPEDs fall enrollment survey with its database of student charges. NCES requires that universities  self-report these figures. The analysis assumes that annual tuition charges can be split down the middle across semesters. 

FURTHER WORK: Filter out part time?

# What this file does - Detailed Methodology

This file reconciles three different databases maintained by the National Council for Education Statistics, available here. All three use provisional data, but the difference between the provisional and final release was understood to be so small that NCES encouraged the use of 2018 data for this project. 

- ```hd2018.csv```, a database that lists schools and their ownership, address and whether or not they're active. I used this to figure out that there were 79 schools in CT, 4 of which were considered to be inactive. In addition, 2 had terminated operations as of 2018.
    - The data dictionary for this file can be viewed at 
    - From this file, I saved the 75 schools to ```ct_universities_2018.csv```
    
    
- ```ic_2018_ay.csv```, from which I was able to pull undergraduate and graduate tuition numbers. When I joined the database with the 79 schools, I was only able to successfully join 73 universities on their UNITID, a unique code determined by NCES' IPEDS Data Center. 
    - The data dictionary can be viewed at
    - On closer inspection, it turns out that of the 6 entries that I couldn't match, 4 were inactive and 2 had closed that year. Dropping these entries therefore aided my analysis. 


- ```ef2018a.csv```, which maintains enrollment numbers by degree level and status by racein Fall 2019. Non-resident alien is considered its own racial category in IPEDS data.
    - The data dictionary can be viewed at

# Index

- Isolating CT universities
- Fall Enrollment Numbers
    - All students
    - Ugrads
    - Grads
- Revenue calculations - The Issue
- Revenue Calculations - The Fix

# Tools

In [1]:
import pandas as pd

# Isolating CT universities

Use ```hd2018.csv``` to find a complete, unique list of universities in CT and to obtain whether or not they are active.

In [ ]:
#read in 

In [3]:
df_ic = pd.read_csv('hd2018.csv', encoding='latin-1')

In [ ]:
#filter for CT

In [6]:
df_ct_total = df_ic[df_ic['STABBR']=='CT']

In [ ]:
#save file

In [41]:
df_ct_total[['UNITID', "INSTNM", "LONGITUD","LATITUDE","OPEID","CYACTIVE","ACT"]].to_csv('ct_universities_2018',index=False)

In [ ]:
#total no of CT universities

In [14]:
len(df_ct_total)

79

In [ ]:
#checking 79 unique entries

In [16]:
df_ct_total['UNITID'].nunique()

79

In [ ]:
#yes, 79 unique entries

In [17]:
df_ct_total['OPEID'].nunique()

79

In [ ]:
#4 inactive

In [18]:
df_ct_total['CYACTIVE'].value_counts(dropna=False)

1    75
3     4
Name: CYACTIVE, dtype: int64

In [ ]:
# +2 deactivated that year

In [141]:
df_ct_total.groupby(by='CYACTIVE')['ACT'].value_counts()

CYACTIVE  ACT
1         A      73
          M       2
3         C       2
          G       2
Name: ACT, dtype: int64

# Fall Enrollment Numbers

In [ ]:
#fall enrollment 

In [12]:
df_fall_enrollment = pd.read_csv('ef2018a.csv')

In [ ]:
#lots of unique entries nationwide

In [47]:
df_fall_enrollment['UNITID'].nunique()

6352

# - Fall Enrollment - All students

Filtering by ```EFALEVEL==1``` and then using the CT universities database to get the CT ones in order to merge with the tuition database.

In [ ]:
#filtering for all studentd

In [20]:
df_fall_enrollment_all_students = df_fall_enrollment[df_fall_enrollment['EFALEVEL']==1]

In [ ]:
#right join for with CT universities database

In [30]:
df_fall_enrollment_all_students_ct = df_fall_enrollment_all_students.merge(df_ct_total, left_on='UNITID', right_on='UNITID', how='right')

In [ ]:
#merge the 79 even if theydon't have enrollment numbers associated

In [34]:
df_fall_enrollment_all_students.merge(df_ct_total, left_on='UNITID', right_on='UNITID')

,UNITID,EFALEVEL,LINE,SECTION,LSTUDY,XEFTOTLT,EFTOTLT,XEFTOTLM,EFTOTLM,XEFTOTLW,...,CBSATYPE,CSA,NECTA,COUNTYCD,COUNTYNM,CNGDSTCD,LONGITUD,LATITUDE,DFRCGID,DFRCUSCG
0,128498,1,29,3,4,R,1456,R,441,R,...,1,408,75700,9009,New Haven County,903,-72.923848,41.332519,131,1
1,128577,1,29,3,4,R,1858,R,1003,R,...,1,278,78100,9003,Hartford County,902,-72.570177,41.998409,91,1
2,128674,1,29,3,4,R,117,R,16,R,...,1,408,75700,9009,New Haven County,903,-72.848201,41.278557,62,2
3,128744,1,29,3,4,R,5485,R,2030,R,...,1,408,71950,9001,Fairfield County,904,-73.190166,41.164741,113,2
4,128771,1,29,3,4,R,11822,R,5908,R,...,1,278,73450,9003,Hartford County,905,-72.765991,41.692502,117,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,480903,1,29,3,4,R,269,R,238,R,...,1,408,71950,9001,Fairfield County,904,-73.179406,41.186706,230,2
69,481571,1,29,3,4,R,8,Z,0,R,...,1,408,78700,9009,New Haven County,905,-73.055819,41.597490,55,2
70,484127,1,29,3,4,R,98,R,23,R,...,1,408,72850,9001,Fairfield County,905,-73.290525,41.392938,33,2
71,485607,1,29,3,4,R,47,R,3,R,...,1,408,78700,9009,New Haven County,905,-72.999969,41.543472,47,2


In [ ]:
#all 79

In [33]:
df_fall_enrollment_all_students_ct

,UNITID,EFALEVEL,LINE,SECTION,LSTUDY,XEFTOTLT,EFTOTLT,XEFTOTLM,EFTOTLM,XEFTOTLW,...,CBSATYPE,CSA,NECTA,COUNTYCD,COUNTYNM,CNGDSTCD,LONGITUD,LATITUDE,DFRCGID,DFRCUSCG
0,128498,1.0,29.0,3.0,4.0,R,1456.0,R,441.0,R,...,1,408,75700,9009,New Haven County,903,-72.923848,41.332519,131,1
1,128577,1.0,29.0,3.0,4.0,R,1858.0,R,1003.0,R,...,1,278,78100,9003,Hartford County,902,-72.570177,41.998409,91,1
2,128674,1.0,29.0,3.0,4.0,R,117.0,R,16.0,R,...,1,408,75700,9009,New Haven County,903,-72.848201,41.278557,62,2
3,128744,1.0,29.0,3.0,4.0,R,5485.0,R,2030.0,R,...,1,408,71950,9001,Fairfield County,904,-73.190166,41.164741,113,2
4,128771,1.0,29.0,3.0,4.0,R,11822.0,R,5908.0,R,...,1,278,73450,9003,Hartford County,905,-72.765991,41.692502,117,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,128683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,278,73450,9003,Hartford County,901,-72.920217,41.632345,-2,-2
75,130448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,408,71950,9001,Fairfield County,904,-73.201106,41.201198,-2,-2
76,460604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,408,71950,9001,Fairfield County,903,-73.130760,41.161240,-2,-2
77,492768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,278,76450,9011,New London County,902,-72.110340,41.452711,-2,-2


In [ ]:
#which ones didn't have fall enrollment numbers associated; looks to be the inactive ones

In [42]:
df_fall_enrollment_all_students_ct[df_fall_enrollment_all_students_ct['EFALEVEL'].isna()][['INSTNM','CYACTIVE','UNITID',"ACT"]]

,INSTNM,CYACTIVE,UNITID,ACT
73,Paul Mitchell the School-Danbury,1,128540,M
74,Lincoln College of New England-Southington,1,128683,M
75,St Vincent's College,3,130448,C
76,Stratford School of Aviation Maintenance Techn...,3,460604,C
77,Three Rivers Community College - Corrigan-Radg...,3,492768,G
78,Three Rivers Community College - York Correcti...,3,492777,G


In [ ]:
#31 colleges and universities in the state reported any intl student enrollment at all

In [55]:
df_fall_enrollment_all_students_ct[df_fall_enrollment_all_students_ct['EFNRALT']>0]

,UNITID,EFALEVEL,LINE,SECTION,LSTUDY,XEFTOTLT,EFTOTLT,XEFTOTLM,EFTOTLM,XEFTOTLW,...,CBSATYPE,CSA,NECTA,COUNTYCD,COUNTYNM,CNGDSTCD,LONGITUD,LATITUDE,DFRCGID,DFRCUSCG
0,128498,1.0,29.0,3.0,4.0,R,1456.0,R,441.0,R,...,1,408,75700,9009,New Haven County,903,-72.923848,41.332519,131,1
1,128577,1.0,29.0,3.0,4.0,R,1858.0,R,1003.0,R,...,1,278,78100,9003,Hartford County,902,-72.570177,41.998409,91,1
3,128744,1.0,29.0,3.0,4.0,R,5485.0,R,2030.0,R,...,1,408,71950,9001,Fairfield County,904,-73.190166,41.164741,113,2
4,128771,1.0,29.0,3.0,4.0,R,11822.0,R,5908.0,R,...,1,278,73450,9003,Hartford County,905,-72.765991,41.692502,117,1
5,128780,1.0,29.0,3.0,4.0,R,1641.0,R,518.0,R,...,1,278,73450,9003,Hartford County,905,-72.767972,41.688246,139,1
7,128902,1.0,29.0,3.0,4.0,R,1844.0,R,714.0,R,...,1,278,76450,9011,New London County,902,-72.107321,41.379833,142,1
9,129020,1.0,29.0,3.0,4.0,R,27412.0,R,13325.0,R,...,1,278,73450,9013,Tolland County,902,-72.249948,41.809098,104,1
10,129154,1.0,29.0,3.0,4.0,R,3397.0,R,677.0,R,...,1,278,73450,9003,Hartford County,901,-72.640320,41.742977,172,1
12,129215,1.0,29.0,3.0,4.0,R,5208.0,R,2150.0,R,...,1,148,73450,9015,Windham County,902,-72.218753,41.721673,134,1
13,129242,1.0,29.0,3.0,4.0,R,5273.0,R,2005.0,R,...,1,408,71950,9001,Fairfield County,904,-73.255903,41.157673,122,1


In [ ]:
#confirming they're all unique

In [68]:
len(df_fall_enrollment_all_students_ct[df_fall_enrollment_all_students_ct['EFNRALT']==0][['INSTNM']])

31

# Tuition and fees - All students

Using ```TUITION3``` and ```TUITION7``` variables for ugrad and grad respectively; just doing the match with all to see the maximum number of values that don't carry over

In [43]:
df_tuition = pd.read_csv('ic2018_ay.csv')

In [48]:
df_tuition['UNITID'].nunique()

4125

In [60]:
df_tuition_ct_all = df_tuition.merge(df_fall_enrollment_all_students_ct,how='right')

In [67]:
len(df_tuition_ct_all[df_tuition_ct_all['TUITION1'].isna()][['INSTNM']])

34

In [70]:
df_tuition_ct_all[(df_tuition_ct_all['TUITION1'].isna())&(df_tuition_ct_all['EFNRALT'].isna())][['INSTNM']]

,INSTNM
74,Paul Mitchell the School-Danbury
75,St Vincent's College
76,Stratford School of Aviation Maintenance Techn...
77,Three Rivers Community College - Corrigan-Radg...
78,Three Rivers Community College - York Correcti...


In [73]:
len(df_tuition_ct_all[(df_tuition_ct_all['TUITION1'].isna())&(df_tuition_ct_all['EFNRALT']==0)][['INSTNM']])

26

In [98]:
df_tuition_ct_all[(df_tuition_ct_all['TUITION1'].isna())&(df_tuition_ct_all['EFNRALT']>0)][['INSTNM','EFNRALT',"CYACTIVE","ACT"]]

,INSTNM,EFNRALT,CYACTIVE,ACT
48,Paul Mitchell the School-North Haven,1.0,1,A
51,New England Tractor Trailer Training School of...,1.0,1,A
67,International Institute of Cosmetology,2.0,1,A


# - Fall Enrollment - Ugrad

In [78]:
df_fall_enrollment_ugrad_students = df_fall_enrollment[df_fall_enrollment['EFALEVEL']==2]

In [79]:
df_fall_enrollment_ugrad_students['UNITID'].nunique()

6053

In [80]:
df_fall_enrollment_ugrad_students.merge(df_ct_total, left_on='UNITID', right_on='UNITID')

,UNITID,EFALEVEL,LINE,SECTION,LSTUDY,XEFTOTLT,EFTOTLT,XEFTOTLM,EFTOTLM,XEFTOTLW,...,CBSATYPE,CSA,NECTA,COUNTYCD,COUNTYNM,CNGDSTCD,LONGITUD,LATITUDE,DFRCGID,DFRCUSCG
0,128498,2,99,3,1,R,1116,R,369,R,...,1,408,75700,9009,New Haven County,903,-72.923848,41.332519,131,1
1,128577,2,99,3,1,R,1858,R,1003,R,...,1,278,78100,9003,Hartford County,902,-72.570177,41.998409,91,1
2,128674,2,99,3,1,R,117,R,16,R,...,1,408,75700,9009,New Haven County,903,-72.848201,41.278557,62,2
3,128744,2,99,3,1,R,3415,R,1216,R,...,1,408,71950,9001,Fairfield County,904,-73.190166,41.164741,113,2
4,128771,2,99,3,1,R,9546,R,5084,R,...,1,278,73450,9003,Hartford County,905,-72.765991,41.692502,117,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,480903,2,99,3,1,R,269,R,238,R,...,1,408,71950,9001,Fairfield County,904,-73.179406,41.186706,230,2
67,481571,2,99,3,1,R,8,Z,0,R,...,1,408,78700,9009,New Haven County,905,-73.055819,41.597490,55,2
68,484127,2,99,3,1,R,98,R,23,R,...,1,408,72850,9001,Fairfield County,905,-73.290525,41.392938,33,2
69,485607,2,99,3,1,R,47,R,3,R,...,1,408,78700,9009,New Haven County,905,-72.999969,41.543472,47,2


In [84]:
df_fall_enrollment_ugrad_students_ct = df_fall_enrollment_ugrad_students.merge(df_ct_total, left_on='UNITID', right_on='UNITID', how='right')

In [85]:
df_fall_enrollment_ugrad_students_ct

,UNITID,EFALEVEL,LINE,SECTION,LSTUDY,XEFTOTLT,EFTOTLT,XEFTOTLM,EFTOTLM,XEFTOTLW,...,CBSATYPE,CSA,NECTA,COUNTYCD,COUNTYNM,CNGDSTCD,LONGITUD,LATITUDE,DFRCGID,DFRCUSCG
0,128498,2.0,99.0,3.0,1.0,R,1116.0,R,369.0,R,...,1,408,75700,9009,New Haven County,903,-72.923848,41.332519,131,1
1,128577,2.0,99.0,3.0,1.0,R,1858.0,R,1003.0,R,...,1,278,78100,9003,Hartford County,902,-72.570177,41.998409,91,1
2,128674,2.0,99.0,3.0,1.0,R,117.0,R,16.0,R,...,1,408,75700,9009,New Haven County,903,-72.848201,41.278557,62,2
3,128744,2.0,99.0,3.0,1.0,R,3415.0,R,1216.0,R,...,1,408,71950,9001,Fairfield County,904,-73.190166,41.164741,113,2
4,128771,2.0,99.0,3.0,1.0,R,9546.0,R,5084.0,R,...,1,278,73450,9003,Hartford County,905,-72.765991,41.692502,117,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,129491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,278,73450,9003,Hartford County,901,-72.707766,41.769861,164,2
75,130448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,408,71950,9001,Fairfield County,904,-73.201106,41.201198,-2,-2
76,460604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,408,71950,9001,Fairfield County,903,-73.130760,41.161240,-2,-2
77,492768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,278,76450,9011,New London County,902,-72.110340,41.452711,-2,-2


In [86]:
df_fall_enrollment_ugrad_students_ct[df_fall_enrollment_ugrad_students_ct['EFALEVEL'].isna()][['INSTNM','CYACTIVE','UNITID',"ACT"]]

,INSTNM,CYACTIVE,UNITID,ACT
71,Paul Mitchell the School-Danbury,1,128540,M
72,Lincoln College of New England-Southington,1,128683,M
73,Rensselaer at Hartford,1,129428,A
74,Hartford Seminary,1,129491,A
75,St Vincent's College,3,130448,C
76,Stratford School of Aviation Maintenance Techn...,3,460604,C
77,Three Rivers Community College - Corrigan-Radg...,3,492768,G
78,Three Rivers Community College - York Correcti...,3,492777,G


In [109]:
df_tuition_ct_ugrad = df_tuition.merge(df_fall_enrollment_ugrad_students_ct,how='right')

In [110]:
df_tuition_ct_ugrad

,UNITID,XTUIT1,TUITION1,XFEE1,FEE1,XHRCHG1,HRCHG1,XTUIT2,TUITION2,XFEE2,...,CBSATYPE,CSA,NECTA,COUNTYCD,COUNTYNM,CNGDSTCD,LONGITUD,LATITUDE,DFRCGID,DFRCUSCG
0,128498,R,31570,R,490,R,1315,R,31570,R,...,1,408,75700,9009,New Haven County,903,-72.923848,41.332519,131,1
1,128577,R,3912,R,552,R,163,R,3912,R,...,1,278,78100,9003,Hartford County,902,-72.570177,41.998409,91,1
2,128683,R,18780,R,1395,R,710,R,18780,R,...,1,278,73450,9003,Hartford County,901,-72.920217,41.632345,-2,-2
3,128744,R,30750,R,2110,R,1025,R,30750,R,...,1,408,71950,9001,Fairfield County,904,-73.190166,41.164741,113,2
4,128771,R,5642,R,4974,R,520,R,5642,R,...,1,278,73450,9003,Hartford County,905,-72.765991,41.692502,117,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,128540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,408,72850,9001,Fairfield County,905,-73.451841,41.396480,-2,-2
75,130448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,408,71950,9001,Fairfield County,904,-73.201106,41.201198,-2,-2
76,460604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,408,71950,9001,Fairfield County,903,-73.130760,41.161240,-2,-2
77,492768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,278,76450,9011,New London County,902,-72.110340,41.452711,-2,-2


In [111]:
df_tuition_ct_ugrad[(df_tuition_ct_ugrad['TUITION1'].isna())&(df_tuition_ct_ugrad['EFNRALT']>0)][['INSTNM','EFNRALT',"CYACTIVE","ACT"]]

,INSTNM,EFNRALT,CYACTIVE,ACT
48,Paul Mitchell the School-North Haven,1.0,1,A
51,New England Tractor Trailer Training School of...,1.0,1,A
67,International Institute of Cosmetology,2.0,1,A


# Fall Enrollment - Graduate Students

In [95]:
df_fall_enrollment_grad_students = df_fall_enrollment[df_fall_enrollment['EFALEVEL']==12]

In [99]:
df_fall_enrollment_grad_students['UNITID'].unique()

array([100654, 100663, 100690, ..., 492607, 492689, 492704])

In [100]:
df_fall_enrollment_grad_students.merge(df_ct_total, left_on='UNITID', right_on='UNITID')

,UNITID,EFALEVEL,LINE,SECTION,LSTUDY,XEFTOTLT,EFTOTLT,XEFTOTLM,EFTOTLM,XEFTOTLW,...,CBSATYPE,CSA,NECTA,COUNTYCD,COUNTYNM,CNGDSTCD,LONGITUD,LATITUDE,DFRCGID,DFRCUSCG
0,128498,12,99,3,3,R,340,R,72,R,...,1,408,75700,9009,New Haven County,903,-72.923848,41.332519,131,1
1,128744,12,99,3,3,R,2070,R,814,R,...,1,408,71950,9001,Fairfield County,904,-73.190166,41.164741,113,2
2,128771,12,99,3,3,R,2276,R,824,R,...,1,278,73450,9003,Hartford County,905,-72.765991,41.692502,117,1
3,128780,12,99,3,3,R,92,R,34,R,...,1,278,73450,9003,Hartford County,905,-72.767972,41.688246,139,1
4,129020,12,99,3,3,R,8279,R,3916,R,...,1,278,73450,9013,Tolland County,902,-72.249948,41.809098,104,1
5,129154,12,99,3,3,R,67,R,17,R,...,1,278,73450,9003,Hartford County,901,-72.640320,41.742977,172,1
6,129215,12,99,3,3,R,195,R,64,R,...,1,148,73450,9015,Windham County,902,-72.218753,41.721673,134,1
7,129242,12,99,3,3,R,1096,R,338,R,...,1,408,71950,9001,Fairfield County,904,-73.255903,41.157673,122,1
8,129428,12,99,3,3,R,67,R,60,R,...,1,278,73450,9003,Hartford County,901,-72.673916,41.773462,176,2
9,129491,12,99,3,3,R,138,R,60,R,...,1,278,73450,9003,Hartford County,901,-72.707766,41.769861,164,2


In [102]:
df_fall_enrollment_grad_students_ct = df_fall_enrollment_grad_students.merge(df_ct_total, left_on='UNITID', right_on='UNITID', how='right')

In [113]:
df_tuition_ct_grad = df_tuition.merge(df_fall_enrollment_grad_students_ct,how='right')

In [115]:
df_tuition_ct_grad[(df_tuition_ct_grad['TUITION1'].isna())&(df_tuition_ct_grad['EFNRALT']>0)][['INSTNM','EFNRALT',"CYACTIVE","ACT"]]

,INSTNM,EFNRALT,CYACTIVE,ACT


# Revenue Calculations - The Issue

In [138]:
df_tuition_ct_grad[df_tuition_ct_grad['TUITION7']=='.'][['INSTNM','EFNRALT']]

,INSTNM,EFNRALT
1,Asnuntuck Community College,NaN
2,Lincoln College of New England-Southington,NaN
7,Porter and Chester Institute of Branford,NaN
12,Capital Community College,NaN
13,Rensselaer at Hartford,0.0
17,Housatonic Community College,NaN
18,Manchester Community College,NaN
19,Naugatuck Valley Community College,NaN
20,Middlesex Community College,NaN
21,Mitchell College,NaN


In [153]:
df_tuition_ct_ugrad[df_tuition_ct_ugrad['TUITION3']=='.'][['INSTNM','EFNRALT']]

,INSTNM,EFNRALT
13,Rensselaer at Hartford,NaN
14,Hartford Seminary,NaN
37,United States Coast Guard Academy,31.0


# Revenue Calculations - The Fix

Ugrad

In [145]:
df_tuition_ct_ugrad_rev = df_tuition_ct_ugrad[(df_tuition_ct_ugrad['TUITION3']!='.')&(df_tuition_ct_ugrad['TUITION3'].isna()==False)]

In [147]:
df_tuition_ct_ugrad_rev['TUITION3'] = df_tuition_ct_ugrad_rev['TUITION3'].astype(float)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [151]:
df_tuition_ct_ugrad_rev['Ugrad_revenue']=df_tuition_ct_ugrad_rev['TUITION3']*df_tuition_ct_ugrad_rev['EFNRALT']

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [152]:
df_tuition_ct_ugrad_rev['Ugrad_revenue'].sum()

203311682.0

Grad

In [154]:
df_tuition_ct_grad_rev = df_tuition_ct_grad[(df_tuition_ct_grad['TUITION7']!='.')&(df_tuition_ct_grad['TUITION7'].isna()==False)]

In [155]:
df_tuition_ct_grad_rev['TUITION7'] = df_tuition_ct_grad_rev['TUITION7'].astype(float)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [157]:
df_tuition_ct_grad_rev['Grad_revenue']=df_tuition_ct_grad_rev['TUITION7']*df_tuition_ct_grad_rev['EFNRALT']

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [158]:
df_tuition_ct_grad_rev['Grad_revenue'].sum()

194785219.0

In [159]:
203311682.0 + 194785219.0

398096901.0